In [1]:
import CropModels
from CropDataset import MyDataSet,normalize_torch,normalize_05,normalize_dataset,preprocess,preprocess_hflip,preprocess_with_augmentation
import pandas as pd
from torch.utils.data import DataLoader
import torch.nn as nn
from tensorboardX import SummaryWriter
import datetime
import os
import torch
from torch.autograd import Variable
from utils import RunningMean
import utils
from PIL import Image
import numpy as np
import random
NB_CLASS=59
SEED=888
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
torch.backends.cudnn.benchmark = True


In [2]:
BATCH_SIZE=32
IMAGE_SIZE=224    # 不同模型修改不同的Size
IMAGE_TRAIN_PRE='../data/AgriculturalDisease_trainingset/images/'
ANNOTATION_TRAIN='../data/AgriculturalDisease_trainingset/AgriculturalDisease_train_annotations_deleteNoise.json' #是否需要剔除两类异常类
IMAGE_VAL_PRE='../data/AgriculturalDisease_validationset/images/'
ANNOTATION_VAL='../data/AgriculturalDisease_validationset/AgriculturalDisease_validation_annotations_deleteNoise.json' #是否需要剔除两类异常类
date=str(datetime.date.today())
with open(ANNOTATION_TRAIN) as datafile1:
    trainDataFram=pd.read_json(datafile1,orient='records')
with open(ANNOTATION_VAL) as datafile2: #first check if it's a valid json file or not
    validateDataFram =pd.read_json(datafile2,orient='records')    
def getmodel():
    print('[+] loading model... ', end='', flush=True)
    model=CropModels.densenet161_finetune(NB_CLASS) # 需要修改为使用的model
    model.cuda()
    print('Done')
    return model
def train(epochNum):
    writer=SummaryWriter('log/'+date+'/DesNet161/') # 创建 /log/日期/InceptionResnet的组织形式  不同模型需要修改不同名称
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    weight=torch.Tensor([1,3,3,3,3,4,2,3,3,3,3,3,3,3,3,3,2,3,3,3,2,3,4,2,3,1,1,3,2,2,1,3,3,1,3,2,3,3,3,3,2,1,3,2,3,3,3,1,3,3,4,4,3,2,2,3,1,1,3]).cuda()
    criterion=nn.CrossEntropyLoss(weight=weight).cuda()
#     lx, px = utils.predict(model,val_dataLoader)
#     min_loss = criterion(Variable(px), Variable(lx)).item()
    min_loss=4.1
    print('min_loss is :%f'%(min_loss))
    min_acc=0.80
    patience=0
    lr=0.0
    momentum=0.9
    for epoch in range(epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if epoch==3 or epoch==4 or epoch==5:
            lr=0.00003
            momentum=0.95
            print('set lr=:%f,momentum=%f'%(lr,momentum))
        if epoch==6:
            lr=1e-4
            momentum=0.9
            print('set lr=:%f,momentum=%f'%(lr,momentum))        
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/DesNet161/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/5
            print('loss has increased lr divide 5 lr now is :%f'%(lr))
        if epoch==0 or epoch==1 or epoch==2: #第一轮首先训练全连接层
            lr=1e-3
            optimizer = torch.optim.Adam(model.fresh_params(),lr = lr,amsgrad=True,weight_decay=1e-4)
#             optimizer=torch.optim.SGD(params=model.fresh_params(),lr=lr,momentum=0.9)

            #optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
            #optimizer=torch.optim.SGD(params=model.fresh_params(),lr=lr,momentum=0.9)
        else:
            optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
            
             #optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
#             optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))       
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'DesNet161', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'DesNet161', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
train(60)

[+] loading model... 

/home/lujunfeng/anaconda3/envs/Conda_Env_Pytorch/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


Done
min_loss is :4.100000
Epoch 0/59
----------
[epoch:0,batch:29]:acc: 0.165625,loss:3.837802
[epoch:0,batch:59]:acc: 0.297396,loss:3.391238
[epoch:0,batch:89]:acc: 0.367708,loss:3.063303
[epoch:0,batch:119]:acc: 0.422396,loss:2.793565
[epoch:0,batch:149]:acc: 0.466667,loss:2.584416
[epoch:0,batch:179]:acc: 0.496007,loss:2.429176
[epoch:0,batch:209]:acc: 0.520387,loss:2.292664
[epoch:0,batch:239]:acc: 0.537370,loss:2.182895
[epoch:0,batch:269]:acc: 0.555093,loss:2.083971
[epoch:0,batch:299]:acc: 0.570937,loss:1.995705
[epoch:0,batch:299]: val_loss:1.077655,val_acc:0.729676,val_total:4539
[epoch:0,batch:329]:acc: 0.584659,loss:1.920064
[epoch:0,batch:359]:acc: 0.593056,loss:1.857804
[epoch:0,batch:389]:acc: 0.602644,loss:1.800236
[epoch:0,batch:419]:acc: 0.611607,loss:1.747029
[epoch:0,batch:449]:acc: 0.621250,loss:1.695188
[epoch:0,batch:479]:acc: 0.626758,loss:1.654603
[epoch:0,batch:509]:acc: 0.631311,loss:1.616521
[epoch:0,batch:539]:acc: 0.636574,loss:1.581834
[epoch:0,batch:569]

[epoch:4,batch:119]:acc: 0.859375,loss:0.355627
[epoch:4,batch:149]:acc: 0.853750,loss:0.358028
[epoch:4,batch:179]:acc: 0.852951,loss:0.363766
[epoch:4,batch:209]:acc: 0.851637,loss:0.374592
[epoch:4,batch:239]:acc: 0.853385,loss:0.372864
[epoch:4,batch:269]:acc: 0.854282,loss:0.372449
[epoch:4,batch:299]:acc: 0.854896,loss:0.372060
[epoch:4,batch:299]: val_loss:0.454710,val_acc:0.834986,val_total:4539
[epoch:4,batch:329]:acc: 0.856155,loss:0.369862
[epoch:4,batch:359]:acc: 0.856510,loss:0.367403
[epoch:4,batch:389]:acc: 0.856410,loss:0.366987
[epoch:4,batch:419]:acc: 0.856696,loss:0.367973
[epoch:4,batch:449]:acc: 0.855764,loss:0.368838
[epoch:4,batch:479]:acc: 0.855924,loss:0.368469
[epoch:4,batch:509]:acc: 0.856005,loss:0.369404
[epoch:4,batch:539]:acc: 0.856539,loss:0.368446
[epoch:4,batch:569]:acc: 0.855263,loss:0.369900
[epoch:4,batch:599]:acc: 0.855729,loss:0.369174
[epoch:4,batch:599]: val_loss:0.438078,val_acc:0.836308,val_total:4539
[epoch:4,batch:629]:acc: 0.856002,loss:0.3

In [4]:
def reuseTrain(path,epochNum):
    writer=SummaryWriter('log/'+date+'/DesNet161/') # 创建 /log/日期/InceptionResnet的组织形式
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess_with_augmentation(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    criterion=nn.CrossEntropyLoss().cuda()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])
    min_loss=modelParams['val_loss']
    print('min_loss is :%f'%(min_loss))   
    min_acc=max(modelParams['val_correct'],0.81)
    print('val_correct is %f'%(min_acc))
    patience=0
    lr=0.00003
    momentum=0.9
    beginepoch=modelParams['epoch']
    for epoch in range(beginepoch,epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if patience==2:
            patience=0
            model.load_state_dict(torch.load('../model/DesNet161/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/10
        optimizer = torch.optim.Adam(model.parameters(),lr = lr,amsgrad=True,weight_decay=1e-4)
        print('lr now is %f'%(lr))
        print('now patience is %d '%(patience))
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))      
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'DesNet161', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'DesNet161', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
reuseTrain('../model/DesNet161/2018-11-01_acc_best.pth',60)

[+] loading model... 

/home/lujunfeng/anaconda3/envs/Conda_Env_Pytorch/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


Done
min_loss is :0.430037
val_correct is 0.844459
Epoch 6/59
----------
lr now is 0.000030
now patience is 0 
[epoch:6,batch:29]:acc: 0.909375,loss:0.241067
[epoch:6,batch:59]:acc: 0.904167,loss:0.250481
[epoch:6,batch:89]:acc: 0.904514,loss:0.240705
[epoch:6,batch:119]:acc: 0.902083,loss:0.242806
[epoch:6,batch:149]:acc: 0.901458,loss:0.242425
[epoch:6,batch:179]:acc: 0.900521,loss:0.243219
[epoch:6,batch:209]:acc: 0.899554,loss:0.245344
[epoch:6,batch:239]:acc: 0.897786,loss:0.248654
[epoch:6,batch:269]:acc: 0.896296,loss:0.252380
[epoch:6,batch:299]:acc: 0.894896,loss:0.255700
[epoch:6,batch:299]: val_loss:0.397014,val_acc:0.852611,val_total:4539
[epoch:6,batch:329]:acc: 0.895833,loss:0.253637
[epoch:6,batch:359]:acc: 0.893663,loss:0.257207
[epoch:6,batch:389]:acc: 0.894231,loss:0.256767
[epoch:6,batch:419]:acc: 0.893155,loss:0.258344
[epoch:6,batch:449]:acc: 0.893403,loss:0.257544
[epoch:6,batch:479]:acc: 0.892448,loss:0.259064
[epoch:6,batch:509]:acc: 0.891789,loss:0.260278
[epoc

[epoch:10,batch:29]:acc: 0.966667,loss:0.117303
[epoch:10,batch:59]:acc: 0.963021,loss:0.124353
[epoch:10,batch:89]:acc: 0.959028,loss:0.132778
[epoch:10,batch:119]:acc: 0.960677,loss:0.130835
[epoch:10,batch:149]:acc: 0.957500,loss:0.133087
[epoch:10,batch:179]:acc: 0.955556,loss:0.136296
[epoch:10,batch:209]:acc: 0.953720,loss:0.137364
[epoch:10,batch:239]:acc: 0.952344,loss:0.137779
[epoch:10,batch:269]:acc: 0.953241,loss:0.136723
[epoch:10,batch:299]:acc: 0.953125,loss:0.136260
[epoch:10,batch:299]: val_loss:0.376543,val_acc:0.863847,val_total:4539
[epoch:10,batch:329]:acc: 0.953409,loss:0.135150
[epoch:10,batch:359]:acc: 0.952865,loss:0.134897
[epoch:10,batch:389]:acc: 0.952804,loss:0.134715
[epoch:10,batch:419]:acc: 0.953125,loss:0.134908
[epoch:10,batch:449]:acc: 0.953472,loss:0.134584
[epoch:10,batch:479]:acc: 0.953581,loss:0.133909
[epoch:10,batch:509]:acc: 0.953922,loss:0.133184
[epoch:10,batch:539]:acc: 0.954109,loss:0.132964
[epoch:10,batch:569]:acc: 0.954386,loss:0.132806


[epoch:14,batch:89]:acc: 0.928125,loss:0.187564
[epoch:14,batch:119]:acc: 0.929948,loss:0.183464
[epoch:14,batch:149]:acc: 0.929583,loss:0.185591
[epoch:14,batch:179]:acc: 0.928299,loss:0.186684
[epoch:14,batch:209]:acc: 0.928869,loss:0.185868
[epoch:14,batch:239]:acc: 0.928516,loss:0.185647
[epoch:14,batch:269]:acc: 0.928472,loss:0.185984
[epoch:14,batch:299]:acc: 0.927813,loss:0.186368
[epoch:14,batch:299]: val_loss:0.382635,val_acc:0.860322,val_total:4539
[epoch:14,batch:329]:acc: 0.927178,loss:0.185940
[epoch:14,batch:359]:acc: 0.925955,loss:0.187493
[epoch:14,batch:389]:acc: 0.925240,loss:0.188513
[epoch:14,batch:419]:acc: 0.925223,loss:0.188277
[epoch:14,batch:449]:acc: 0.926389,loss:0.185967
[epoch:14,batch:479]:acc: 0.925586,loss:0.186388
[epoch:14,batch:509]:acc: 0.925858,loss:0.185765
[epoch:14,batch:539]:acc: 0.925347,loss:0.186683
[epoch:14,batch:569]:acc: 0.925329,loss:0.186867
[epoch:14,batch:599]:acc: 0.925729,loss:0.186247
[epoch:14,batch:599]: val_loss:0.374961,val_acc

...

[epoch:30,batch:449]:acc: 0.922708,loss:0.190006
[epoch:30,batch:479]:acc: 0.922070,loss:0.190954
[epoch:30,batch:509]:acc: 0.922181,loss:0.190865
[epoch:30,batch:539]:acc: 0.922512,loss:0.190321
[epoch:30,batch:569]:acc: 0.922314,loss:0.190958
[epoch:30,batch:599]:acc: 0.922813,loss:0.190600
[epoch:30,batch:599]: val_loss:0.374017,val_acc:0.860542,val_total:4539
[epoch:30,batch:629]:acc: 0.922222,loss:0.191580
[epoch:30,batch:659]:acc: 0.922396,loss:0.192410
[epoch:30,batch:689]:acc: 0.922645,loss:0.192019
[epoch:30,batch:719]:acc: 0.922266,loss:0.192013
[epoch:30,batch:749]:acc: 0.921958,loss:0.192240
[epoch:30,batch:779]:acc: 0.922276,loss:0.192385
[epoch:30,batch:809]:acc: 0.922492,loss:0.192139
[epoch:30,batch:839]:acc: 0.922582,loss:0.191702
[epoch:30,batch:869]:acc: 0.922881,loss:0.191378
[epoch:30,batch:899]:acc: 0.923056,loss:0.190892
[epoch:30,batch:899]: val_loss:0.376285,val_acc:0.860983,val_total:4539
[epoch:30,batch:929]:acc: 0.923522,loss:0.190119
[epoch:30,batch:959]:ac

[epoch:34,batch:569]:acc: 0.923739,loss:0.188724
[epoch:34,batch:599]:acc: 0.923594,loss:0.189752
[epoch:34,batch:599]: val_loss:0.375891,val_acc:0.861643,val_total:4539
[epoch:34,batch:629]:acc: 0.924157,loss:0.188861
[epoch:34,batch:659]:acc: 0.923816,loss:0.189308
[epoch:34,batch:689]:acc: 0.923551,loss:0.189815
[epoch:34,batch:719]:acc: 0.923047,loss:0.190547
[epoch:34,batch:749]:acc: 0.923167,loss:0.190437
[epoch:34,batch:779]:acc: 0.923197,loss:0.190270
[epoch:34,batch:809]:acc: 0.923727,loss:0.189407
[epoch:34,batch:839]:acc: 0.923884,loss:0.189024
[epoch:34,batch:869]:acc: 0.923707,loss:0.189481
[epoch:34,batch:899]:acc: 0.923507,loss:0.189569
[epoch:34,batch:899]: val_loss:0.377183,val_acc:0.861423,val_total:4539
[epoch:34,batch:929]:acc: 0.923387,loss:0.189450
[epoch:34,batch:959]:acc: 0.923177,loss:0.189896
[epoch:34,batch:989]:acc: 0.923548,loss:0.189507
[epoch:34] :acc: 0.923606,loss:0.189339,lr:0.000000,patience:1
[epoch:34]: val_loss:0.377849,val_acc:0.862304,
Epoch 35/5

[epoch:38,batch:659]:acc: 0.923201,loss:0.191014
[epoch:38,batch:689]:acc: 0.922871,loss:0.191796
[epoch:38,batch:719]:acc: 0.923090,loss:0.191846
[epoch:38,batch:749]:acc: 0.923208,loss:0.191644
[epoch:38,batch:779]:acc: 0.923758,loss:0.191011
[epoch:38,batch:809]:acc: 0.923765,loss:0.190734
[epoch:38,batch:839]:acc: 0.923884,loss:0.190701
[epoch:38,batch:869]:acc: 0.923994,loss:0.190622
[epoch:38,batch:899]:acc: 0.923819,loss:0.190623
[epoch:38,batch:899]: val_loss:0.375975,val_acc:0.862304,val_total:4539
[epoch:38,batch:929]:acc: 0.924026,loss:0.190222
[epoch:38,batch:959]:acc: 0.924089,loss:0.189945
[epoch:38,batch:989]:acc: 0.924621,loss:0.189467
[epoch:38] :acc: 0.924583,loss:0.189703,lr:0.000000,patience:1
[epoch:38]: val_loss:0.376066,val_acc:0.862745,
Epoch 39/59
----------
lr now is 0.000000
now patience is 0 
[epoch:39,batch:29]:acc: 0.926042,loss:0.178980
[epoch:39,batch:59]:acc: 0.921354,loss:0.190619
[epoch:39,batch:89]:acc: 0.920486,loss:0.191602
[epoch:39,batch:119]:acc

[epoch:42,batch:749]:acc: 0.924167,loss:0.189492
[epoch:42,batch:779]:acc: 0.924519,loss:0.189663
[epoch:42,batch:809]:acc: 0.924961,loss:0.189242
[epoch:42,batch:839]:acc: 0.925074,loss:0.189256
[epoch:42,batch:869]:acc: 0.925575,loss:0.188682
[epoch:42,batch:899]:acc: 0.926111,loss:0.188112
[epoch:42,batch:899]: val_loss:0.377907,val_acc:0.862525,val_total:4539
[epoch:42,batch:929]:acc: 0.926109,loss:0.188274
[epoch:42,batch:959]:acc: 0.926400,loss:0.187452
[epoch:42,batch:989]:acc: 0.925852,loss:0.188199
[epoch:42] :acc: 0.925876,loss:0.188210,lr:0.000000,patience:0
[epoch:42]: val_loss:0.374539,val_acc:0.862745,
Epoch 43/59
----------
lr now is 0.000000
now patience is 1 
[epoch:43,batch:29]:acc: 0.918750,loss:0.190381
[epoch:43,batch:59]:acc: 0.923958,loss:0.188419
[epoch:43,batch:89]:acc: 0.927083,loss:0.185450
[epoch:43,batch:119]:acc: 0.925781,loss:0.187398
[epoch:43,batch:149]:acc: 0.927083,loss:0.185919
[epoch:43,batch:179]:acc: 0.925174,loss:0.187805
[epoch:43,batch:209]:acc

[epoch:46,batch:869]:acc: 0.923276,loss:0.190412
[epoch:46,batch:899]:acc: 0.923681,loss:0.190040
[epoch:46,batch:899]: val_loss:0.375196,val_acc:0.862304,val_total:4539
[epoch:46,batch:929]:acc: 0.923454,loss:0.189781
[epoch:46,batch:959]:acc: 0.923372,loss:0.189690
[epoch:46,batch:989]:acc: 0.922443,loss:0.190385
[epoch:46] :acc: 0.922439,loss:0.190475,lr:0.000000,patience:0
[epoch:46]: val_loss:0.377956,val_acc:0.860322,
Epoch 47/59
----------
lr now is 0.000000
now patience is 1 
[epoch:47,batch:29]:acc: 0.915625,loss:0.195762
[epoch:47,batch:59]:acc: 0.913021,loss:0.203071
[epoch:47,batch:89]:acc: 0.917361,loss:0.191414
[epoch:47,batch:119]:acc: 0.920833,loss:0.190482
[epoch:47,batch:149]:acc: 0.919167,loss:0.192601
[epoch:47,batch:179]:acc: 0.918924,loss:0.192807
[epoch:47,batch:209]:acc: 0.920387,loss:0.191108
[epoch:47,batch:239]:acc: 0.920443,loss:0.190544
[epoch:47,batch:269]:acc: 0.920486,loss:0.191709
[epoch:47,batch:299]:acc: 0.919792,loss:0.194773
[epoch:47,batch:299]: va

In [3]:
def TrainWithRawData(path,epochNum):
    writer=SummaryWriter('log/'+date+'/DesNet161/') # 创建 /log/日期/InceptionResnet的组织形式
    train_dataset=MyDataSet(json_Description=ANNOTATION_TRAIN,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_TRAIN_PRE)
    val_dataset=MyDataSet(json_Description=ANNOTATION_VAL,transform=preprocess(normalize_torch,IMAGE_SIZE),path_pre=IMAGE_VAL_PRE)
    train_dataLoader=DataLoader(dataset=train_dataset,batch_size=BATCH_SIZE,num_workers=16,shuffle=True)
    val_dataLoader=DataLoader(dataset=val_dataset,batch_size=BATCH_SIZE,num_workers=1,shuffle=False)
    model=getmodel()
    criterion=nn.CrossEntropyLoss().cuda()
    modelParams=torch.load(path)
    model.load_state_dict(modelParams['state_dict'])
    min_loss=modelParams['val_loss']
    print('min_loss is :%f'%(min_loss))
    print('val_correct is %f'%(modelParams['val_correct']))
    min_acc=max(modelParams['val_correct'],0.81)
    optinizerSave=modelParams['optimizer']
    patience=0
    lr=1e-4
    momentum=0.9
    beginepoch=modelParams['epoch']
    for epoch in range(beginepoch,epochNum):
        print('Epoch {}/{}'.format(epoch, epochNum - 1))
        print('-' * 10)
        if patience==3:
            patience=0
            model.load_state_dict(torch.load('../model/DesNet161/'+date+'_loss_best.pth')['state_dict'])
            lr=lr/5
            print('lr desencd')
        if epoch==beginepoch:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
#             optimizer.load_state_dict(optinizerSave)
#             lr=optimizer['lr']
#             momentum=optimizer['momentum']
            print('begin lr is ',lr)
            
        else:
            optimizer=torch.optim.SGD(params=model.parameters(),lr=lr,momentum=momentum)
                   
        running_loss = RunningMean()
        running_corrects = RunningMean()
        for batch_idx, (inputs, labels) in enumerate(train_dataLoader):
            model.train(True)
            n_batchsize=inputs.size(0)
            inputs = Variable(inputs).cuda()
            labels = Variable(labels).cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            if isinstance(outputs,tuple):
                loss=sum((criterion(o,labels)) for o in outputs)
            else:
                loss = criterion(outputs, labels)
            running_loss.update(loss.item(),1)
            running_corrects.update(torch.sum(preds == labels.data).data,n_batchsize)
            loss.backward()
            optimizer.step()
            if batch_idx%30==29:
                print('[epoch:%d,batch:%d]:acc: %f,loss:%f'%(epoch,batch_idx,running_corrects.value,running_loss.value))
                if batch_idx%300==299: 
                    niter = epoch * len(train_dataset)/BATCH_SIZE + batch_idx
                    writer.add_scalar('Train/Acc',running_corrects.value,niter)
                    writer.add_scalar('Train/Loss',running_loss.value,niter)
                    lx,px=utils.predict(model,val_dataLoader)
                    log_loss = criterion(Variable(px), Variable(lx))
                    log_loss = log_loss.item()
                    _, preds = torch.max(px, dim=1)
                    accuracy = torch.mean((preds == lx).float())
                    writer.add_scalar('Val/Acc',accuracy,niter)
                    writer.add_scalar('Val/Loss',log_loss,niter)
                    print('[epoch:%d,batch:%d]: val_loss:%f,val_acc:%f,val_total:%d'%(epoch,batch_idx,log_loss,accuracy,len(val_dataset)))
                    if  log_loss < min_loss:
                        utils.snapshot('../model/', 'DesNet161', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy })          

                        min_loss=log_loss
                        print('save new model loss,now loss is ',min_loss)

                    if accuracy>min_acc:
                        utils.snapshot('../model/', 'DesNet161', {
                               'epoch': epoch + 1,
                               'state_dict': model.state_dict(),
                               'optimizer': optimizer.state_dict(),
                               'val_loss': log_loss,
                               'val_correct':accuracy },key='acc') 
                        min_acc=accuracy
                        print('save new model acc,now acc is ',min_acc)
        print('[epoch:%d] :acc: %f,loss:%f,lr:%f,patience:%d'%(epoch,running_corrects.value,running_loss.value,lr,patience))         
        lx,px=utils.predict(model,val_dataLoader)
        log_loss = criterion(Variable(px), Variable(lx))
        log_loss = log_loss.item()
        _, preds = torch.max(px, dim=1)
        accuracy = torch.mean((preds == lx).float())
        writer.add_scalar('Val/Acc',accuracy,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        writer.add_scalar('Val/Loss',log_loss,(epoch+1) * len(train_dataset)/BATCH_SIZE)
        print('[epoch:%d]: val_loss:%f,val_acc:%f,'%(epoch,log_loss,accuracy))
        if  log_loss < min_loss:
            utils.snapshot('../model/', 'DesNet161', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy })          
            patience = 0
            min_loss=log_loss
            print('save new model loss,now loss is ',min_loss)
        else:
            patience += 1
        if accuracy>min_acc:
            utils.snapshot('../model/', 'DesNet161', {
                   'epoch': epoch + 1,
                   'state_dict': model.state_dict(),
                   'optimizer': optimizer.state_dict(),
                   'val_loss': log_loss,
                   'val_correct':accuracy },key='acc') 
            min_acc=accuracy
            print('save new model acc,now acc is ',min_acc)

In [ ]:
TrainWithRawData('../model/DesNet161/2018-11-01_acc_best.pth',60)

[+] loading model... 

/home/lujunfeng/anaconda3/envs/Conda_Env_Pytorch/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


Done
min_loss is :0.382938
val_correct is 0.865389
Epoch 12/59
----------
begin lr is  0.0001
[epoch:12,batch:29]:acc: 0.944792,loss:0.134083
[epoch:12,batch:59]:acc: 0.941146,loss:0.144881
[epoch:12,batch:89]:acc: 0.942014,loss:0.143863
[epoch:12,batch:119]:acc: 0.941927,loss:0.145871
[epoch:12,batch:149]:acc: 0.941875,loss:0.146085
[epoch:12,batch:179]:acc: 0.941840,loss:0.146879
[epoch:12,batch:209]:acc: 0.939435,loss:0.148838
[epoch:12,batch:239]:acc: 0.940755,loss:0.148168
[epoch:12,batch:269]:acc: 0.939815,loss:0.149295
[epoch:12,batch:299]:acc: 0.939583,loss:0.150478
[epoch:12,batch:299]: val_loss:0.372221,val_acc:0.869134,val_total:4539
save new model loss,now loss is  0.3722209632396698
save new model acc,now acc is  tensor(0.8691, device='cuda:0')
[epoch:12,batch:329]:acc: 0.939962,loss:0.150594
[epoch:12,batch:359]:acc: 0.939670,loss:0.150664
[epoch:12,batch:389]:acc: 0.939904,loss:0.150077
[epoch:12,batch:419]:acc: 0.938765,loss:0.150779
[epoch:12,batch:449]:acc: 0.938958,l

save new model acc,now acc is  tensor(0.8707, device='cuda:0')
[epoch:15,batch:929]:acc: 0.959341,loss:0.118820
[epoch:15,batch:959]:acc: 0.959473,loss:0.118541
[epoch:15,batch:989]:acc: 0.959470,loss:0.118535
[epoch:15] :acc: 0.959422,loss:0.118795,lr:0.000020,patience:0
[epoch:15]: val_loss:0.367276,val_acc:0.870456,
Epoch 16/59
----------
[epoch:16,batch:29]:acc: 0.965625,loss:0.101848
[epoch:16,batch:59]:acc: 0.963021,loss:0.108099
[epoch:16,batch:89]:acc: 0.960764,loss:0.112451
[epoch:16,batch:119]:acc: 0.960156,loss:0.113375
[epoch:16,batch:149]:acc: 0.959167,loss:0.114261
[epoch:16,batch:179]:acc: 0.957986,loss:0.115825
[epoch:16,batch:209]:acc: 0.957589,loss:0.117024
[epoch:16,batch:239]:acc: 0.957682,loss:0.117070
[epoch:16,batch:269]:acc: 0.957986,loss:0.117376
[epoch:16,batch:299]:acc: 0.958750,loss:0.117106
[epoch:16,batch:299]: val_loss:0.368306,val_acc:0.869354,val_total:4539
[epoch:16,batch:329]:acc: 0.959564,loss:0.116206
[epoch:16,batch:359]:acc: 0.959462,loss:0.116014

[epoch:19]: val_loss:0.369849,val_acc:0.868694,
Epoch 20/59
----------
[epoch:20,batch:29]:acc: 0.954167,loss:0.126430
[epoch:20,batch:59]:acc: 0.952083,loss:0.129708
[epoch:20,batch:89]:acc: 0.953472,loss:0.125645
[epoch:20,batch:119]:acc: 0.958333,loss:0.119517
[epoch:20,batch:149]:acc: 0.958958,loss:0.119525
[epoch:20,batch:179]:acc: 0.958160,loss:0.121246
[epoch:20,batch:209]:acc: 0.958482,loss:0.120624
[epoch:20,batch:239]:acc: 0.958333,loss:0.120137
[epoch:20,batch:269]:acc: 0.958912,loss:0.119471
[epoch:20,batch:299]:acc: 0.958125,loss:0.119634
[epoch:20,batch:299]: val_loss:0.368250,val_acc:0.870676,val_total:4539
[epoch:20,batch:329]:acc: 0.958617,loss:0.119016
[epoch:20,batch:359]:acc: 0.957812,loss:0.120113
[epoch:20,batch:389]:acc: 0.957772,loss:0.120056
[epoch:20,batch:419]:acc: 0.958333,loss:0.119202
[epoch:20,batch:449]:acc: 0.959097,loss:0.117667
[epoch:20,batch:479]:acc: 0.958984,loss:0.118482
[epoch:20,batch:509]:acc: 0.959252,loss:0.117996
[epoch:20,batch:539]:acc: 0